In [ ]:
#r "nuget: Plotly.NET, 2.0.0-preview.6"
#r "nuget: Plotly.NET.Interactive, 2.0.0-preview.6"
#r "nuget: Microsoft.Data.Analysis, 0.18.0"

open System.IO
open System.Diagnostics
open Microsoft.Data.Analysis
open Plotly.NET

let parseMilliseconds (str: string) =
    if str.Contains(" ms") then
        Single.Parse(str.Replace(" ms", ""))
    elif str.Contains(" s") then
        Single.Parse(str.Replace(" s", "")) * 1000.f
    else
        failwith "Invalid string"

let parseAllocated (str: string) =
    if str.Contains(" MB") then
        Single.Parse(str.Replace(" MB", ""))
    elif str.Contains(" KB") then
        Single.Parse(str.Replace(" KB", "")) / 1024.f
    else
        failwith "Invalid string"

let run name args workingDir =
    let info = ProcessStartInfo()
    info.Arguments <- args
    info.FileName <- name
    info.UseShellExecute <- false
    info.WorkingDirectory <- workingDir
    let p = Process.Start(info)
    p.WaitForExit()
    if p.ExitCode <> 0 then
        failwith $"{name} {args} failed."

let resultsPath = Path.Combine(__SOURCE_DIRECTORY__, "BenchmarkDotNet.Artifacts\\results\\BenchmarkComparison.TypeCheckingBenchmark1-report.csv")

let benchmarkCurrent(): string * DataFrame =
    printfn "Benchmarking Current (Today)..."
    run "dotnet" "run -c Release --project run_current.fsproj" __SOURCE_DIRECTORY__
    let df = DataFrame.LoadCsv(resultsPath)
    printfn "Current (Today) Done"
    ("Current (Today)", df)

let benchmarkVersion (name: string) (version: string) (constants: string): string * DataFrame =
    try File.Delete(Path.Combine(__SOURCE_DIRECTORY__, "setup.fsproj")) with | _ -> ()
    try
        printfn $"Benchmarking {name}..."
        let setupTemplate = File.ReadAllText(Path.Combine(__SOURCE_DIRECTORY__, "setup_version_template.fsproj"))
        let setup = setupTemplate.Replace("{TEMPLATE_FCS_VERSION}", version).Replace("{TEMPLATE_DEFINE_CONSTANTS}", constants)
        File.WriteAllText(Path.Combine(__SOURCE_DIRECTORY__, "setup.fsproj"), setup)
        run "dotnet" "run -c Release --project run.fsproj" __SOURCE_DIRECTORY__

        let df = DataFrame.LoadCsv(resultsPath)
        printfn $"{name} Done"
        (name, df)
    finally
        try File.Delete(Path.Combine(__SOURCE_DIRECTORY__, "setup.fsproj")) with | _ -> ()

let benchmarkCommit (commitHash: string) (constants: string): string * DataFrame =
    try File.Delete(Path.Combine(__SOURCE_DIRECTORY__, "setup.fsproj")) with | _ -> ()
    let tmp = Path.GetTempFileName()
    try File.Delete(tmp) with | _ -> ()
    let tmpDir = Path.Combine(Path.GetDirectoryName(tmp), Path.GetFileNameWithoutExtension(tmp))
    Directory.CreateDirectory(tmpDir) |> ignore

    try
        let fcsPath = Path.Combine(tmpDir, "src/fsharp/FSharp.Compiler.Service/FSharp.Compiler.Service.fsproj")
        let fcsOutputPath = Path.Combine(tmpDir, "artifacts/bin/FSharp.Compiler.Service/Release/netstandard2.0/FSharp.Compiler.Service.dll")
        printfn $"Cloning 'dotnet/fsharp.git' in '{tmpDir}'..."
        run "git" $"clone https://github.com/dotnet/fsharp.git {tmpDir}" __SOURCE_DIRECTORY__
        printfn $"Switching to '{commitHash}'..."
        run "git" $"reset --hard {commitHash}" tmpDir
        printfn "Building fsharp..."
        run "cmd" $"/C build.cmd -c Release" tmpDir
        
        printfn $"Benchmarking {commitHash}..."
        let setupTemplate = File.ReadAllText(Path.Combine(__SOURCE_DIRECTORY__, "setup_commit_template.fsproj"))
        let setup = setupTemplate.Replace("{TEMPLATE_FCS_PATH}", fcsOutputPath).Replace("{TEMPLATE_DEFINE_CONSTANTS}", constants)
        File.WriteAllText(Path.Combine(__SOURCE_DIRECTORY__, "setup.fsproj"), setup)
        run "dotnet" "run -c Release --project run.fsproj" __SOURCE_DIRECTORY__

        let df = DataFrame.LoadCsv(resultsPath)
        printfn $"{commitHash} Done"
        (commitHash, df)
    finally
        try File.Delete(Path.Combine(__SOURCE_DIRECTORY__, "setup.fsproj")) with | _ -> ()
        try Directory.Delete(tmpDir) with | _ -> ()

In [ ]:
let benchmarkData =
    [
        // Note: SERVICE_30_0_0 and SERVICE_13_0_0 refer to define constants in order to build the benchmark on older versions.
        benchmarkCurrent()
      //  benchmarkCommit "ffbc0d410c776ad108811dcc9f01f3185e9a6e92" ""
        benchmarkVersion "40.0.0 (6/22/2021)" "40.0.0" ""
        benchmarkVersion "35.0.0 (4/10/2020)" "35.0.0" "SERVICE_30_0_0"
        benchmarkVersion "30.0.0 (6/29/2019)" "30.0.0" "SERVICE_30_0_0"
        benchmarkVersion "25.0.1 (9/5/2018)" "25.0.1" "SERVICE_13_0_0"
        benchmarkVersion "20.0.1 (2/21/2018)" "20.0.1" "SERVICE_13_0_0"
        benchmarkVersion "13.0.0 (6/28/2017)" "13.0.0" "SERVICE_13_0_0"
    ]

In [ ]:
// Data cleanup

let df =
    (benchmarkData.[0] |> snd, benchmarkData.[1..])
    ||> List.fold (fun df (_, df2) ->
        df.Append(df2.Rows)
    )

let x =
    benchmarkData
    |> List.map (fun (name, _) -> name)

let meanColumn = df.Columns.["Mean"]
let allocatedColumn = df.Columns.["Allocated"]

let y1 =
    [
        for i = 0L to meanColumn.Length - 1L do
            meanColumn.[i] :?> string
            |> parseMilliseconds
    ]
let meanData = (x, y1) ||> List.zip |> List.rev
let y2 =
    [
        for i = 0L to allocatedColumn.Length - 1L do
            allocatedColumn.[i] :?> string
            |> parseAllocated
    ]
let allocatedData = (x, y2) ||> List.zip |> List.rev

// Charts

let meanLine = Chart.Line(meanData, Name="Mean (ms)")
let allocatedLine = Chart.Line(allocatedData, Name="Allocated (MB)")

Chart.Combine([meanLine;allocatedLine])